In [1]:
!pwd

/home/sagemaker-user/Brain-Tumor-Classification-MRI/notebooks


In [2]:
%cd ..

/home/sagemaker-user/Brain-Tumor-Classification-MRI


# Evaluation Configuration

In [13]:
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

In [25]:
@dataclass
class EvalConfig:
    model_path: str
    testing_data: str
    params_image_size: list
    params_batch_size: int

# Configuration Manager

In [9]:
from src.constants import *
from src.utils import read_yaml, create_dirs
from src import logger
from src.config import ConfigManager
import os

In [24]:
class ConfigManager(ConfigManager):
    def get_eval_config(self):
        return EvalConfig(
            model_path = os.path.join(self.artifacts_folder, self.config.training.trained_model_path),
            testing_data = os.path.join(self.artifacts_folder, self.config.data.data_folder_name, 'Testing'),
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

# Eval Function

In [15]:
import tensorflow as tf
import json

In [22]:
class Eval:
    def __init__(self, config: EvalConfig):
        self.config = config

    def _data_generator(self):
        generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
        self.generator = generator.flow_from_directory(
            directory = self.config.testing_data,
            shuffle = False,
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
        )

    def eval(self, path):
        model = tf.keras.models.load_model(self.config.model_path)
        self._data_generator()
        score = model.evaluate(self.generator)
        data = {
            'loss': score[0],
            'accuracy': score[1]
        }
        with open(path, "w") as f:
            json.dump(data, f, indent=4)
            logger.info(f"\nEvaluation file saved at: {path}\n")

# Perform The Step

In [23]:
try:
    config = ConfigManager().get_eval_config()
    evaluate = Eval(config)
    evaluate.eval('artifacts/eval.json')
except Exception as e:
    raise e

[2024-09-28 17:50:58,547: INFO: utils: The file: config.yaml loaded successfully...]
[2024-09-28 17:50:58,549: INFO: utils: The file: params.yaml loaded successfully...]
Found 394 images belonging to 4 classes.
13/13 [==============================] - 13s 947ms/step - loss: 1.3883 - accuracy: 0.2538
[2024-09-28 17:51:12,249: INFO: 2468986840: 
Evaluation file saved at: artifacts/eval.json
]
